<center><h1>Segmenting and Clustering Neighborhoods in Toronto</h1></center>

<h2>Contents</h2>

Hi There. This notebook is divided into 3 sections and are as follows:
<ul>
    <li>Web Scraping Toronto Neighborhood Data</li>
    <li>Grabbing coordinates of Neighborhoods</li>
    <li>Analysis of Neighborhoods</li>
</ul>
        

<h3> Part 1: Web Scraping Toronto Neighborhood Data</h3>

<h5>Required Libraries</h5>
<ul>
    <li>Selenium</li>
    <li>Beautiful Soup</li>
    <li>Pandas</li>
    </u>

In [8]:
# Installing Selenium
import os
os.system('pip3 install selenium')

0

In [10]:
# Installing BeautifulSoup
os.system('pip3 install BeautifulSoup4')

0

In [9]:
# Installing Pandas if not present
os.system('pip3 install pandas')

0

In [1]:
# Importing Installed Libraries
import pandas as pd
from bs4 import BeautifulSoup
import requests
import os

In [2]:
# pulling Wikipedia page
wiki_page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
wiki_page

<Response [200]>

In [3]:
# Parsing the page with BeautifulSoup
soup = BeautifulSoup(wiki_page.content, 'html.parser')

In [4]:
# Extracting children of wiki_page
wiki_page_children = list(soup.children)
wiki_table = soup.findAll('table','wikitable')

In [38]:
os.system('pip3 install lxml')

0

<b>Reading Data present in the table into a DataFrame</b>

In [5]:
# Reading Html Table Data into DataFrame
data_frame = pd.read_html(str(wiki_table))[0]

In [6]:
data_frame.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


<b>Cleaning the DataFrame based on below rules:</b>
<ul>
    <li>The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood.</li>
    <li>Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    <li>More than one neighborhood can exist in one postal code area.</li>
    <li>These rows will be combined into one row with the neighborhoods separated with a comma.</li>
    <li>If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.</li>
</ul>

In [7]:
# Get names of indexes for which column Borourgh is Not Assigned
indexNames = data_frame[ data_frame['Borough'] == 'Not assigned' ].index
 
# Delete these row indexes from dataFrame
data_frame.drop(indexNames , inplace=True)
data_frame.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [8]:
# Grouping by postal code
grouped_data = data_frame.groupby(['Postal code'])
grouped_data = grouped_data.apply(pd.DataFrame)
grouped_data['Neighborhood'] = grouped_data['Neighborhood'].str.replace('/',',')
grouped_data['Neighborhood'] = grouped_data['Neighborhood'].str.replace(" , ", ",")
grouped_data.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park,Harbourfront"
5,M6A,North York,"Lawrence Manor,Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park,Ontario Provincial Government"


In [9]:
# Reseting Index
grouped_data = grouped_data.reset_index()
grouped_data.drop('index', axis=1, inplace=True)
grouped_data.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park,Harbourfront"
3,M6A,North York,"Lawrence Manor,Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park,Ontario Provincial Government"


In [16]:
grouped_data.shape

(103, 3)

<br/>
<h3> Part 2: Grabbing coordinates of Neighborhoods</h3>

<h5> Grabbing Latitude and Longitude Coordinates for the postal codes </h5>
<h6> Directly using CSV from http://cocl.us/Geospatial_data as geocoder is not functioning as expected.</h6>

In [11]:
lat_long_coords = pd.read_csv('Geospatial_Coordinates.csv')
lat_long_coords.head()
lat_long_coords['Postal code']=lat_long_coords['Postal Code']
lat_long_coords.drop('Postal Code', axis=1, inplace=True)
lat_long_coords.head()

,Latitude,Longitude,Postal code
0,43.806686,-79.194353,M1B
1,43.784535,-79.160497,M1C
2,43.763573,-79.188711,M1E
3,43.770992,-79.216917,M1G
4,43.773136,-79.239476,M1H


<h4>Grabbing relavant latitude and Longitude Coordinates from above csv data</h4>

In [12]:
joined_data = pd.merge(grouped_data, lat_long_coords, on='Postal code', how='inner')

In [13]:
joined_data.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park,Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor,Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park,Ontario Provincial Government",43.662301,-79.389494


In [14]:
joined_data.shape

(103, 5)

<br/>
<h3> Part 3: Analysis of Neighborhoods</h3>

<h5> Grabbing Latitude and Longitude Coordinates for the postal codes </h5>
<h6> Directly using CSV from http://cocl.us/Geospatial_data as geocoder is not functioning as expected.</h6>

In [17]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [19]:
# Making a copy of joined_data
dataframe = pd.DataFrame(joined_data)
dataframe.drop('Postal code', axis=1, inplace=True)
dataframe.head()

,Borough,Neighborhood,Latitude,Longitude
0,North York,Parkwoods,43.753259,-79.329656
1,North York,Victoria Village,43.725882,-79.315572
2,Downtown Toronto,"Regent Park,Harbourfront",43.654260,-79.360636
3,North York,"Lawrence Manor,Lawrence Heights",43.718518,-79.464763
4,Downtown Toronto,"Queen's Park,Ontario Provincial Government",43.662301,-79.389494


<h5> Create a Map of Toronto </h5>

In [23]:
latitude = 43.651070
longitude = -79.347015
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(dataframe['Latitude'], dataframe['Longitude'], dataframe['Borough'], dataframe['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [26]:
# @hidden_cell

CLIENT_ID = 'YNFFXY301FYO5F1NIG35ZGXG05UPXNPFAMY4Y33IPWCHICOE' # your Foursquare ID
CLIENT_SECRET = 'WVPVSIXZPHWZSAQ5JWWPGLL3Z5XDQIMQ0GSIKGEK2KO0AHLZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [27]:
dataframe.loc[0, 'Neighborhood']

'Parkwoods'

In [28]:
neighborhood_latitude = dataframe.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = dataframe.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = dataframe.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


In [30]:
# Creating URL

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=YNFFXY301FYO5F1NIG35ZGXG05UPXNPFAMY4Y33IPWCHICOE&client_secret=WVPVSIXZPHWZSAQ5JWWPGLL3Z5XDQIMQ0GSIKGEK2KO0AHLZ&v=20180605&ll=43.7532586,-79.3296565&radius=500&limit=100'

In [35]:
from pandas.io.json import json_normalize

results = requests.get(url).json()

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-35-e66964be8255>:20: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [47]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [52]:
toronto_venues = getNearbyVenues(names=dataframe['Neighborhood'],
                                   latitudes=dataframe['Latitude'],
                                   longitudes=dataframe['Longitude']
                                  )


Parkwoods
Victoria Village
Regent Park,Harbourfront
Lawrence Manor,Lawrence Heights
Queen's Park,Ontario Provincial Government
Islington Avenue
Malvern,Rouge
Don Mills
Parkview Hill,Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park,Princess Gardens,Martin Grove,Islington,Cloverdale
Rouge Hill,Port Union,Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate,Bloordale Gardens,Old Burnhamthorpe,Markland Wood
Guildwood,Morningside,West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor,Wilson Heights,Downsview North
Thorncliffe Park
Richmond,Adelaide,King
Dufferin,Dovercourt Village
Scarborough Village
Fairview,Henry Farm,Oriole
Northwood Park,York University
East Toronto
Harbourfront East,Union Station,Toronto Islands
Little Portugal,Trinity
Kennedy Park,Ionview,East Birchmount Park
Bayview Village
Downsview
The Danforth West,Riverdale
Toronto Dominion Centr

In [94]:
print(toronto_venues.shape)
toronto_venues.head()

(2143, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [95]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [119]:
toronto_onehot.shape

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

toronto_grouped.shape

num_top_venues = 5
 
import numpy as np
    
for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')
    
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

----Agincourt----
                       venue  freq
0                     Lounge  0.25
1               Skating Rink  0.25
2             Breakfast Spot  0.25
3  Latin American Restaurant  0.25
4              Metro Station  0.00


----Alderwood,Long Branch----
          venue  freq
0   Pizza Place  0.22
1          Pool  0.11
2  Skating Rink  0.11
3           Gym  0.11
4      Pharmacy  0.11


----Bathurst Manor,Wilson Heights,Downsview North----
            venue  freq
0            Bank  0.10
1     Coffee Shop  0.10
2       Gift Shop  0.05
3   Shopping Mall  0.05
4  Sandwich Place  0.05


----Bayview Village----
                 venue  freq
0                 Café  0.25
1                 Bank  0.25
2   Chinese Restaurant  0.25
3  Japanese Restaurant  0.25
4          Yoga Studio  0.00


----Bedford Park,Lawrence Manor East----
                venue  freq
0  Italian Restaurant  0.09
1         Coffee Shop  0.09
2          Restaurant  0.09
3      Sandwich Place  0.09
4       Women's Store  0.

         venue  freq
0  Coffee Shop  0.15
1         Park  0.07
2          Pub  0.07
3       Bakery  0.07
4      Theater  0.04


----Richmond,Adelaide,King----
         venue  freq
0  Coffee Shop  0.09
1         Café  0.05
2   Restaurant  0.04
3        Hotel  0.03
4          Gym  0.03


----Rosedale----
                venue  freq
0                Park  0.50
1               Trail  0.25
2          Playground  0.25
3   Mobile Phone Shop  0.00
4  Miscellaneous Shop  0.00


----Roselawn----
                        venue  freq
0                        Pool  0.33
1     Health & Beauty Service  0.33
2                      Garden  0.33
3               Metro Station  0.00
4  Modern European Restaurant  0.00


----Rouge Hill,Port Union,Highland Creek----
                        venue  freq
0  Construction & Landscaping   0.5
1                         Bar   0.5
2                 Yoga Studio   0.0
3               Metro Station   0.0
4  Modern European Restaurant   0.0


----Runnymede,Swansea----
  

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Skating Rink,Lounge,Breakfast Spot,Women's Store,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant
1,"Alderwood,Long Branch",Pizza Place,Coffee Shop,Pharmacy,Sandwich Place,Pub,Pool,Skating Rink,Gym,Construction & Landscaping,Department Store
2,"Bathurst Manor,Wilson Heights,Downsview North",Bank,Coffee Shop,Sushi Restaurant,Bridal Shop,Diner,Restaurant,Fried Chicken Joint,Pizza Place,Pharmacy,Deli / Bodega
3,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Women's Store,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
4,"Bedford Park,Lawrence Manor East",Sandwich Place,Italian Restaurant,Restaurant,Coffee Shop,Women's Store,Thai Restaurant,Comfort Food Restaurant,Juice Bar,Butcher,Café


<h4> k-means Clustering </h4>


In [120]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=5).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [121]:
# add clustering labels
# neighborhoods_venues_sorted.drop('Cluster Labels', axis=1, inplace=True)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = dataframe

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,Parkwoods,43.753259,-79.329656,0.0,Park,Food & Drink Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Falafel Restaurant
1,North York,Victoria Village,43.725882,-79.315572,1.0,Coffee Shop,Pizza Place,Portuguese Restaurant,Hockey Arena,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
2,Downtown Toronto,"Regent Park,Harbourfront",43.654260,-79.360636,1.0,Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Theater,Café,Health Food Store,Historic Site,Hotel
3,North York,"Lawrence Manor,Lawrence Heights",43.718518,-79.464763,1.0,Clothing Store,Women's Store,Furniture / Home Store,Coffee Shop,Boutique,Shoe Store,Miscellaneous Shop,Event Space,Vietnamese Restaurant,Accessories Store
4,Downtown Toronto,"Queen's Park,Ontario Provincial Government",43.662301,-79.389494,1.0,Coffee Shop,Sushi Restaurant,Diner,Yoga Studio,Creperie,Spa,Beer Bar,Sandwich Place,Burger Joint,Burrito Place


In [123]:
print(toronto_merged['Cluster Labels'].unique())

# Correcting Cluster Labels

# filling nulls with 5
toronto_merged['Cluster Labels'].fillna(4, inplace=True)

# converting to int
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype('int32')

[0 1 4 2 3]


<h4> Clustering </h4>

In [124]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [125]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Parkwoods,Park,Food & Drink Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Falafel Restaurant
16,Humewood-Cedarvale,Park,Field,Hockey Arena,Trail,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Department Store
21,Caledonia-Fairbanks,Park,Pool,Women's Store,Airport,Dessert Shop,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
32,Scarborough Village,Women's Store,Playground,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
35,East Toronto,Park,Coffee Shop,Convenience Store,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Dumpling Restaurant
61,Lawrence Park,Park,Swim School,Bus Line,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Falafel Restaurant
64,Weston,Park,Convenience Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
66,York Mills West,Park,Bank,Convenience Store,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
68,Forest Hill North & West,Trail,Park,Sushi Restaurant,Jewelry Store,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
83,"Moore Park,Summerhill East",Park,Playground,Trail,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Empanada Restaurant,Donut Shop,Deli / Bodega


In [101]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Parkwoods,Park,Food & Drink Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Falafel Restaurant
16,Humewood-Cedarvale,Park,Field,Hockey Arena,Trail,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Department Store
21,Caledonia-Fairbanks,Park,Pool,Women's Store,Airport,Dessert Shop,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
35,East Toronto,Park,Coffee Shop,Convenience Store,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Dumpling Restaurant
61,Lawrence Park,Park,Swim School,Bus Line,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Falafel Restaurant
64,Weston,Park,Convenience Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
66,York Mills West,Park,Bank,Convenience Store,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
68,Forest Hill North & West,Trail,Park,Sushi Restaurant,Jewelry Store,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
83,"Moore Park,Summerhill East",Park,Playground,Trail,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Empanada Restaurant,Donut Shop,Deli / Bodega
85,"Milliken,Agincourt North,Steeles East,L'Amorea...",Park,Coffee Shop,Playground,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant


In [126]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,"York Mills,Silver Hills",Martial Arts Dojo,Women's Store,Dessert Shop,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop


In [127]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,"Humberlea,Emery",Food Service,Baseball Field,Women's Store,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Farmers Market
101,"Old Mill South,King's Mill Park,Sunnylea,Humbe...",Locksmith,Baseball Field,Women's Store,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant


In [128]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Islington Avenue,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,"West Deane Park,Princess Gardens,Martin Grove,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,"Willowdale,Newtonbrook",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,Upper Rouge,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
